**1.Data Collection**

**1.1. Imports**

In [96]:
import pandas as pd
import numpy as np


**1.2.Load Data**

In [70]:
asthma_adults= pd.read_csv("https://raw.githubusercontent.com/Mary080102/Capston-Two/main/Raw%20data/NYC%20EH%20Data%20Portal%20-%20Asthma%20emergency%20department%20visits%20(adults)%20(full%20table).csv")
asthma_age_4= pd.read_csv("https://raw.githubusercontent.com/Mary080102/Capston-Two/main/Raw%20data/NYC%20EH%20Data%20Portal%20-%20Asthma%20emergency%20department%20visits%20(age%204%20and%20under)%20(full%20table).csv")
asthma_age5_17=pd.read_csv("https://raw.githubusercontent.com/Mary080102/Capston-Two/main/Raw%20data/NYC%20EH%20Data%20Portal%20-%20Asthma%20emergency%20department%20visits%20(age%205%20to%2017)%20(full%20table).csv")
pm2_5=pd.read_csv("https://raw.githubusercontent.com/Mary080102/Capston-Two/main/Raw%20data/NYC%20EH%20Data%20Portal%20-%20Fine%20particles%20(PM%202.5)%20(full%20table).csv")
no2=pd.read_csv("https://raw.githubusercontent.com/Mary080102/Capston-Two/main/Raw%20data/NYC%20EH%20Data%20Portal%20-%20Nitrogen%20dioxide%20(NO2)%20(full%20table).csv")

In [147]:
print(asthma_adults["Age-adjusted rate per 10,000"].str.contains('†').sum())

1


In [146]:
print(asthma_adults["GeoID"].duplicated().sum())


727


**1.3.Data Joining**

In [71]:
# finding common column to merge
common_columns= set(asthma_adults.columns)& set(asthma_age_4.columns)&set(asthma_age5_17.columns)&set(pm2_5.columns)&set(no2.columns)
print(common_columns)

{'GeoID', 'Geography', 'TimePeriod', 'GeoRank', 'GeoType'}


In [72]:
print(no2.head())

            TimePeriod GeoType  GeoID  GeoRank  \
0  Annual Average 2022      CD    101        6   
1  Annual Average 2022      CD    102        6   
2  Annual Average 2022      CD    103        6   
3  Annual Average 2022      CD    104        6   
4  Annual Average 2022      CD    105        6   

                             Geography  10th percentile ppb  \
0             Financial District (CD1)                 19.3   
1     Greenwich Village and Soho (CD2)                 19.2   
2  Lower East Side and Chinatown (CD3)                 15.8   
3            Clinton and Chelsea (CD4)                 18.6   
4                        Midtown (CD5)                 22.4   

   90th percentile ppb  Mean ppb  
0                 22.4      21.0  
1                 21.1      20.2  
2                 19.9      17.8  
3                 24.4      21.4  
4                 27.8      25.5  


**Because of large dataset that use too much memory, I need to reduce data size by dropping unnecessary columns and filter out unnecessary rows before merging or processing.**

**1.3.1. Dropping Unnecessay Columns:**

In [73]:
# List the columns that want to drop
columns_to_drop=["GeoType","GeoRank"]


In [74]:
## Create a new DataFrame with only the necessary columns
asthma_age_4_sub=asthma_age_4.drop(columns_to_drop,axis=1)

In [75]:
print(asthma_age_4_sub.columns)

Index(['TimePeriod', 'GeoID', 'Geography', 'Estimated annual rate per 10,000',
       'Number'],
      dtype='object')


In [76]:
asthma_age5_17_sub= asthma_age5_17.drop(columns_to_drop,axis=1)
pm2_5_sub= pm2_5.drop(columns_to_drop,axis=1)
no2_sub=no2.drop(columns_to_drop,axis=1)


**1.3.2. Dropping Unnecessay Rows:**

I want to choose three neighborhoods in New York City to assess air quality that can provide an overview of the whole area requires considering diversity in geography, urban density, and potential sources of pollution. Based on the Geography column in csv files that includes all neighborhoods,I want to choose New yourk, Manhatan,Staten Island.

In [77]:
# List of names I want to keep
rows_to_keep=["New York City","Manhattan","Staten Island"]


In [78]:
# Creat a new Dataframe with only rows for our study with the name,"New York City","Manhattan","Staten Island"
asthma_adults_df=asthma_adults.loc[asthma_adults["Geography"].isin(rows_to_keep)]

In [79]:
print(asthma_adults_df.head())

     TimePeriod   GeoType  GeoID  GeoRank      Geography  \
103        2020   Borough      3        1      Manhattan   
105        2020   Borough      5        1  Staten Island   
106        2020  Citywide      1        0  New York City   
210        2019   Borough      3        1      Manhattan   
212        2019   Borough      5        1  Staten Island   

    Age-adjusted rate per 10,000 Estimated annual rate per 10,000  Number  
103                         44.7                             43.5   6,290  
105                         35.1                             33.2   1,287  
106                         48.4                             47.7  33,238  
210                         73.5                             71.1  10,543  
212                         55.4                             52.3   2,066  


In [80]:
asthma_age_4_df= asthma_age_4_sub.loc[asthma_age_4_sub["Geography"].isin(rows_to_keep)]
asthma_age5_17_df=asthma_age5_17_sub.loc[asthma_age5_17_sub["Geography"].isin(rows_to_keep)]
pm2_5_df= pm2_5_sub.loc[pm2_5_sub["Geography"].isin(rows_to_keep)]
no2_df= no2_sub.loc[no2_sub["Geography"].isin(rows_to_keep)]

In [138]:
print(asthma_adults_df['Estimated annual rate per 10,000'].unique())

['43.5' '33.2' '47.7' '71.1' '52.3' '77.1' '84.0' '60.2' '88.4' '92.5'
 '91.7' '55.5' '98.1' '†' '102.4' '71.3' '114.3' '97.7' '71.6' '113.7'
 '104.9' '84.3' '118.2' '101.7' '109.6' '97.0' '76.5' '107.4' '107.6'
 '79.5' '109.1' '108.5' '72.8' '109.0' '148.5' '63.1' '105.0' '144.6'
 '64.3' '103.5' '137.1' '100.3']


In [139]:
print(asthma_adults_df['Estimated annual rate per 10,000'].str.contains('†').sum())

1


**1.3.3.Joining Data**

In [81]:
# rename the common columns except 'GeoID' to merge dataframes
asthma_adults_df= asthma_adults_df.rename(columns={'Geography':'Geography_as','TimePeriod':'TimePeriod_as'})

asthma_age_4_df= asthma_age_4_df.rename(columns={'Geography':'Geography_age4','TimePeriod':'TimePeriod_age4'})

asthma_age5_17_df=asthma_age5_17_df.rename(columns={'Geography':'Geography_age5-17','TimePeriod':'TimePeriod_age5-17'})

pm2_5_df=pm2_5_df.rename(columns={'Geography':'Geography_pm_2_5','TimePeriod':'TimePeriod_pm_2_5'})

no2_df=no2_df.rename(columns={'Geography':'Geography_no2','TimePeriod':'TimePeriod_no2'})


In [82]:
# merge filtered DataFrames
merge1= pd.merge(asthma_adults_df,asthma_age_4_df, on="GeoID", how="inner")

In [83]:
merge2= pd.merge(merge1,asthma_age5_17_df, on="GeoID", how="inner")

In [84]:
merge3= pd.merge(merge2,pm2_5_df, on="GeoID", how="inner")

In [85]:
merge4=pd.merge(merge3,no2_df,on="GeoID", how="inner")

In [86]:
print(merge4.head())

   TimePeriod_as  GeoType  GeoID  GeoRank Geography_as  \
0           2020  Borough      3        1    Manhattan   
1           2020  Borough      3        1    Manhattan   
2           2020  Borough      3        1    Manhattan   
3           2020  Borough      3        1    Manhattan   
4           2020  Borough      3        1    Manhattan   

  Age-adjusted rate per 10,000 Estimated annual rate per 10,000_x Number_x  \
0                         44.7                               43.5    6,290   
1                         44.7                               43.5    6,290   
2                         44.7                               43.5    6,290   
3                         44.7                               43.5    6,290   
4                         44.7                               43.5    6,290   

   TimePeriod_age4 Geography_age4  ...    TimePeriod_pm_2_5 Geography_pm_2_5  \
0             2020      Manhattan  ...  Annual Average 2022        Manhattan   
1             2020    

In [87]:
print(merge4.shape)

(18338544, 26)


**2.Data Definition**

In [88]:
#Using the info method on merge4 to see a summary of the data
merge4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18338544 entries, 0 to 18338543
Data columns (total 26 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   TimePeriod_as                       int64  
 1   GeoType                             object 
 2   GeoID                               int64  
 3   GeoRank                             int64  
 4   Geography_as                        object 
 5   Age-adjusted rate per 10,000        object 
 6   Estimated annual rate per 10,000_x  object 
 7   Number_x                            object 
 8   TimePeriod_age4                     int64  
 9   Geography_age4                      object 
 10  Estimated annual rate per 10,000_y  object 
 11  Number_y                            object 
 12  TimePeriod_age5-17                  int64  
 13  Geography_age5-17                   object 
 14  Estimated annual rate per 10,000    object 
 15  Number                              object 
 16

Based on the results of .info(), some of columns datatype must be changed to int or float. "Age-adjusted rate per 10,000", "Estimated annual rate per 10,000_x","Number_x", "Estimated annual rate per 10,000_y", "Number_y","Estimated annual rate per 10,000", and "Number".



In [127]:
print(merge4['Estimated annual rate per 10,000_x'].unique())
     

['43.5' '71.1' '84.0' '91.7' '102.4' '97.7' '104.9' '101.7' '97.0' '107.6'
 '108.5' '148.5' '144.6' '137.1' '33.2' '52.3' '60.2' '55.5' '71.3' '71.6'
 '84.3' '77.1' '76.5' '79.5' '72.8' '63.1' '64.3' '47.7' '88.4' '92.5'
 '98.1' '†' '114.3' '113.7' '118.2' '109.6' '107.4' '109.1' '109.0'
 '105.0' '103.5' '100.3']


In [132]:
contain_symbol_1= merge4["Estimated annual rate per 10,000_x"].str.contains('†').sum()

In [133]:
print(contain_symbol_1)

451584


In [134]:
print(merge4["Age-adjusted rate per 10,000"].unique())

['44.7' '73.5' '86.8' '95.5' '106.6' '101.6' '109.0' '105.1' '100.0'
 '111.0' '111.3' '150.8' '146.3' '138.3' '35.1' '55.4' '64.0' '59.1'
 '73.7' '73.3' '85.9' '78.0' '78.1' '80.4' '72.8' '63.8' '64.7' '60.3'
 '48.4' '78.3' '89.9' nan '99.1' '†' '114.7' '114.0' '118.3' '109.5'
 '107.2' '108.7' '108.5' '104.4' '102.7' '99.5']


In [135]:
contain_symbol2= merge4["Age-adjusted rate per 10,000"].str.contains('†').sum()

In [136]:
print(contain_symbol2)

451584


In [108]:
# to change data type of "Number_x", because of  contains a comma, first of all need to removve comma
merge4["Number"]=merge4["Number"].str.replace(",","").astype(int)

In [112]:
merge4["Number_y"]=merge4["Number_y"].str.replace(",", "").astype(int)

In [124]:
print(merge4["Number_x"].unique())

['6,290' '10,543' '11,725' '12,872' '14,294' '13,586' '14,537' '13,959'
 '13,121' '14,493' '14,596' '19,862' '19,238' '18,146' '1,287' '2,066'
 '2,246' '2,062' '2,626' '2,625' '3,069' '2,801' '2,755' '2,843' '2,580'
 '2,212' '2,236' '2,085' '33,238' '54,166' '58,907' '61,909' '66,229' '†'
 '76,396' '75,473' '77,792' '71,351' '69,536' '69,944' '69,081' '65,905'
 '64,534' '62,425']


In [125]:
merge4["Number_x"]= merge4["Number_x"].str.replace(",", "")

In [117]:
# "Number_x" column contain '†' character.At first need to  identifying the rows with the '†' character
contain_symbol= merge4["Number_x"].astype(str).str.contains('†').sum()

In [121]:
print(contain_symbol)

451584


In [26]:
#Counts and Percents of Unique Values
unique_counts= merge4.nunique()
print(unique_counts)

TimePeriod_as                          16
GeoType                                 2
GeoID                                   3
GeoRank                                 2
Geography_as                            3
Age-adjusted rate per 10,000           43
Estimated annual rate per 10,000_x     42
Number_x                               44
TimePeriod_age4                        16
Geography_age4                          3
Estimated annual rate per 10,000_y     46
Number_y                               46
TimePeriod_age5-17                     16
Geography_age5-17                       3
Estimated annual rate per 10,000       46
Number                                 46
TimePeriod_pm_2_5                      42
Geography_pm_2_5                        3
10th percentile mcg/m3                 60
90th percentile mcg/m3                 66
Mean mcg/m3                            63
TimePeriod_no2                         42
Geography_no2                           3
10th percentile ppb               

In [44]:
# Rename some columns
rename_dict={"Number_x":"asthma_adults_num","Number_y":"asthma_age_4_num","Number":"asthma_age5_17_num",
            "Mean mcg/m3":"pm2.5_mean_mcg/m3","Mean ppb":"no2_mean_ppb"}

# Rename the columns using the dictionary
merge4.rename(columns=rename_dict, inplace=True)

In [52]:
print(merge4.columns)

Index(['TimePeriod_as', 'GeoType', 'GeoID', 'GeoRank', 'Geography_as',
       'Age-adjusted rate per 10,000', 'Estimated annual rate per 10,000_x',
       'asthma_adults_num', 'TimePeriod_age4', 'Geography_age4',
       'Estimated annual rate per 10,000_y', 'asthma_age_4_num',
       'TimePeriod_age5-17', 'Geography_age5-17',
       'Estimated annual rate per 10,000', 'asthma_age5_17_num',
       'TimePeriod_pm_2_5', 'Geography_pm_2_5', '10th percentile mcg/m3',
       '90th percentile mcg/m3', 'pm2.5_mean_mcg/m3', 'TimePeriod_no2',
       'Geography_no2', '10th percentile ppb', '90th percentile ppb',
       'no2_mean_ppb'],
      dtype='object')


In [45]:
print(merge4["TimePeriod_as"].value_counts().head())

2020    1245384
2019    1245384
2018    1245384
2016    1245384
2014    1245384
Name: TimePeriod_as, dtype: int64


In [46]:
print(merge4["TimePeriod_age4"].value_counts().head())

2020    1192464
2019    1192464
2018    1192464
2016    1192464
2015    1192464
Name: TimePeriod_age4, dtype: int64


In [47]:
print(merge4["TimePeriod_age5-17"].value_counts().head())

2020    1192464
2019    1192464
2018    1192464
2016    1192464
2015    1192464
Name: TimePeriod_age5-17, dtype: int64


In [48]:
print(merge4["TimePeriod_pm_2_5"].value_counts().head())

Annual Average 2022    436632
Summer 2012            436632
Winter 2014-15         436632
Annual Average 2014    436632
Summer 2014            436632
Name: TimePeriod_pm_2_5, dtype: int64


In [49]:
print(merge4["TimePeriod_no2"].value_counts().head())

Annual Average 2022    436632
Summer 2012            436632
Winter 2014-15         436632
Annual Average 2014    436632
Summer 2014            436632
Name: TimePeriod_no2, dtype: int64


Based on the results of value_counts method TimePeriod column has two different data type. The Asthma datas have int type,no2 and pm2.5 have object type. This difference is because of object type contains values that are categorica(like "Annual Average 2022") and the int one include "2028".For the purpose of facilitating chronological comparisons or time series analysis, it is necessary to standardize the data type of the entire TimePeriod column to integer.

In [57]:
merge4[["asthma_adults_num","asthma_age_4_num","asthma_age5_17_num"]].describe()

,asthma_adults_num,asthma_age_4_num,asthma_age5_17_num
count,18338544,18338544,18338544
unique,44,46,46
top,"62,425","19,927","25,834"
freq,451584,451584,451584


In [58]:
merge4[["pm2.5_mean_mcg/m3","no2_mean_ppb"]].describe()

,pm2.5_mean_mcg/m3,no2_mean_ppb
count,1.833854e+07,1.833854e+07
mean,9.051630e+00,1.945028e+01
std,2.077847e+00,6.599239e+00
min,5.200000e+00,7.700000e+00
25%,7.400000e+00,1.390000e+01
50%,8.800000e+00,1.890000e+01
75%,1.040000e+01,2.370000e+01
max,1.530000e+01,3.840000e+01


 **2.1.Categorical Features**

In [65]:
merge4.select_dtypes(include=['object'])
# some of columns type must be changed before this step

,GeoType,Geography_as,"Age-adjusted rate per 10,000","Estimated annual rate per 10,000_x",asthma_adults_num,Geography_age4,"Estimated annual rate per 10,000_y",asthma_age_4_num,Geography_age5-17,"Estimated annual rate per 10,000",asthma_age5_17_num,TimePeriod_pm_2_5,Geography_pm_2_5,TimePeriod_no2,Geography_no2
0,Borough,Manhattan,44.7,43.5,"6,290",Manhattan,58.6,454,Manhattan,65.9,"1,088",Annual Average 2022,Manhattan,Annual Average 2022,Manhattan
1,Borough,Manhattan,44.7,43.5,"6,290",Manhattan,58.6,454,Manhattan,65.9,"1,088",Annual Average 2022,Manhattan,Summer 2022,Manhattan
2,Borough,Manhattan,44.7,43.5,"6,290",Manhattan,58.6,454,Manhattan,65.9,"1,088",Annual Average 2022,Manhattan,Winter 2021-22,Manhattan
3,Borough,Manhattan,44.7,43.5,"6,290",Manhattan,58.6,454,Manhattan,65.9,"1,088",Annual Average 2022,Manhattan,Annual Average 2021,Manhattan
4,Borough,Manhattan,44.7,43.5,"6,290",Manhattan,58.6,454,Manhattan,65.9,"1,088",Annual Average 2022,Manhattan,Summer 2021,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18338539,Citywide,New York City,99.5,100.3,"62,425",New York City,377.2,"19,927",New York City,199.2,"25,834",Winter 2008-09,New York City,Summer 2010,New York City
18338540,Citywide,New York City,99.5,100.3,"62,425",New York City,377.2,"19,927",New York City,199.2,"25,834",Winter 2008-09,New York City,Winter 2009-10,New York City
18338541,Citywide,New York City,99.5,100.3,"62,425",New York City,377.2,"19,927",New York City,199.2,"25,834",Winter 2008-09,New York City,Annual Average 2009,New York City
18338542,Citywide,New York City,99.5,100.3,"62,425",New York City,377.2,"19,927",New York City,199.2,"25,834",Winter 2008-09,New York City,Summer 2009,New York City


**3.Data Cleaning**

**3.1.Number Of Missing Values By Column**

In [59]:
missing=(merge4.isna().sum()/len(merge4))*100

In [61]:
print(missing)

TimePeriod_as                         0.000000
GeoType                               0.000000
GeoID                                 0.000000
GeoRank                               0.000000
Geography_as                          0.000000
Age-adjusted rate per 10,000          2.462486
Estimated annual rate per 10,000_x    0.000000
asthma_adults_num                     0.000000
TimePeriod_age4                       0.000000
Geography_age4                        0.000000
Estimated annual rate per 10,000_y    0.000000
asthma_age_4_num                      0.000000
TimePeriod_age5-17                    0.000000
Geography_age5-17                     0.000000
Estimated annual rate per 10,000      0.000000
asthma_age5_17_num                    0.000000
TimePeriod_pm_2_5                     0.000000
Geography_pm_2_5                      0.000000
10th percentile mcg/m3                0.000000
90th percentile mcg/m3                0.000000
pm2.5_mean_mcg/m3                     0.000000
TimePeriod_no

**3.2.Find Duplicate Rows**

In [62]:
duplicates= merge4.duplicated().sum()

In [64]:
print(duplicates)

0
